# Inputs

In [1]:
import tensorflow as tf

x = tf.placeholder(dtype=tf.float32, name='x')
y = tf.placeholder(dtype=tf.float32, name='y')

x, y

(<tf.Tensor 'x:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'y:0' shape=<unknown> dtype=float32>)

# Variables

Since `w` and `b` are learnable parameters that we will be optimizing to minimize a loss, we can ask TensorFlow to keep track of their values and update them as opposed to doing it *outside* of tensorflow (i.e. in `numpy` land). This is beneficial both for efficiency and conceptual clarity.

Because `w` and `b` are variable we have to tell TensorFlow how to initialize them. For simplicity we will initialize them to `0` with `tf.zeros()`.

In [2]:
w = tf.Variable(initial_value=tf.zeros(shape=[]), name='w')
b = tf.Variable(initial_value=tf.zeros(shape=[]), name='b')

w, b

(<tensorflow.python.ops.variables.Variable at 0x12066da58>,
 <tensorflow.python.ops.variables.Variable at 0x12066d4e0>)

# Add Dense Layer

In [3]:
with tf.name_scope('Dense'):
    z = tf.multiply(x, w, name='z')

y_pred = tf.add(z, b, name='y_pred')

z, y_pred

(<tf.Tensor 'Dense/z:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'y_pred:0' shape=<unknown> dtype=float32>)

# Add Squared Loss

In [4]:
with tf.name_scope('SquaredLoss'):
    r = tf.subtract(y_pred, y, name='r')

loss = tf.square(r, name='loss')
loss

<tf.Tensor 'loss:0' shape=<unknown> dtype=float32>

# Dump Graph

In [5]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter("tensorboard/Variables", sess.graph)
writer.flush()

# Get Gradients

In [6]:
with tf.name_scope('Gradients'):
    dw = tf.gradients(loss, w, name='dw')[0]
    db = tf.gradients(loss, b, name='db')[0]

# SGD Updates

We want to update `w` and `b` inside of TensorFlow via SGD. To accomplish this we will create a placeholder tensor to hold the SGD learning rate and define two more tensors that will modify the values of `w` and `b` with `dw`, `db`, and `lr` via the TensorFlow op `tf.assign_sub()`.

In [7]:
lr = tf.placeholder(dtype=tf.float32, name='lr')

with tf.name_scope('Updates'):
    update_dw = tf.assign_sub(w, tf.multiply(lr, dw), name='update_dw')
    update_db = tf.assign_sub(b, tf.multiply(lr, db), name='update_db')
    
update_dw, update_db

(<tf.Tensor 'Updates/update_dw:0' shape=() dtype=float32_ref>,
 <tf.Tensor 'Updates/update_db:0' shape=() dtype=float32_ref>)

# Initialize Variables

In order to actually initialize the variables in our computational graph, we need to create a tensor which does that. Luckily TensorFlow provides a function for creating such a tensor. Let's creat it and run it.

In [8]:
w.eval()

FailedPreconditionError: Attempting to use uninitialized value w
	 [[Node: _send_w_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=6111275017789653456, tensor_name="w:0", _device="/job:localhost/replica:0/task:0/cpu:0"](w)]]

In [9]:
init = tf.global_variables_initializer()
sess.run(init)

In [10]:
w.eval()

0.0

# Dump Graph

In [11]:
writer = tf.summary.FileWriter("tensorboard/LR with Updates", sess.graph)
writer.flush()

# SGD Step

Now we can run the computational graph which will perform one SGD step with `dw` and `db`. Note that we need to supply `update_dw` and `update_db` to `fetches` in order for the update to actually take place.

In [25]:
lr_ = 0.01
x_ = y_ = 1

values = sess.run(fetches=[loss, update_dw, update_db], feed_dict={x: x_, y: y_, lr: lr_})

{key: value for key, value in zip(['loss', 'w', 'b'], values)}

{'b': 0.21766333, 'loss': 0.34598088, 'w': 0.21766333}